In [2]:
from datetime import datetime
from strategy.stratfact import StratFact
from backtester.backtester import Backtester

In [3]:
start = datetime(2016,1,1)
end = datetime(2017,1,1)

In [4]:
rp = StratFact.create_strat(start,end,"rolling_percent",params= {"timeframe":"daily"
                ,"requirement":3
                ,"days":7
                ,"value":True
                ,"currency":"crypto"})
rp.subscribe()
rpt = Backtester.equity_timeseries_backtest(start,end,5,rp)
rpt

rolling_percent_sim: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 12.74it/s]


In [6]:
pr = StratFact.create_strat(start,end,"progress_report",params={"timeframe":"quarterly"
                    ,"requirement":10})
pr.subscribe()
prt = Backtester.equity_timeseries_backtest(start,end,5,pr)
prt

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single posi

,date,adjclose,ticker,seat,sell_date,sell_price,timeframe,requirement
0,2016-01-22,18.067424,COG,0,2016-02-22,19.874166,quarterly,10
1,2016-02-23,22.699554,NEM,0,2016-04-07,24.969509,quarterly,10
2,2016-04-08,34.823333,EW,0,2016-08-18,38.305667,quarterly,10
3,2016-08-19,7.620000,AMD,0,2016-11-17,8.382000,quarterly,10
4,2016-11-18,14.289700,KEY,0,2017-02-14,15.718670,quarterly,10
...,...,...,...,...,...,...,...,...
79,2020-01-15,61.681675,LEN,4,2020-02-07,67.849843,quarterly,10
80,2020-02-10,38.610000,ENPH,4,2020-02-19,42.471000,quarterly,10
81,2020-02-20,19.981462,NLOK,4,2020-05-29,21.979609,quarterly,10
82,2020-06-01,55.640000,RCL,4,2020-06-05,61.204000,quarterly,10


In [4]:
fp = StratFact.create_strat(start,end,"financial_predict",{})
fp.subscribe()
fpb = Backtester.equity_timeseries_backtest(start,end,5,fp)
fpb

backtesting_seats: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [05:52<00:00, 70.47s/it]


""


In [5]:
spec = StratFact.create_strat(start,end,"speculation",{})
spec.subscribe()
sb = Backtester.equity_timeseries_backtest(start,end,5,spec)
sb

backtesting_seats: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.73s/it]


""
